In [ ]:
import torch
import torchaudio
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nnAudio import Spectrogram

In [ ]:
def extract_mfcc_transformer(sample_rate, n_mfcc=13):
    """
    创建并返回MFCC变换对象。
    
    MFCC（Mel-frequency Cepstral Coefficients）是在梅尔频谱基础上
    通过离散余弦变换（DCT）得到的倒谱系数，广泛用于语音识别和音频分析。
    MFCC能够有效地表示音频信号的频谱包络特征。
    
    Args:
        sample_rate (int): 音频采样率，决定了频谱分析的频率范围
        n_mfcc (int): MFCC系数的数量，通常取12-13个
    
    Returns:
        Spectrogram.MFCC: 初始化完成的MFCC对象
        
    参数说明:
        - sr: 采样率
        - n_mfcc: MFCC系数数量，决定输出特征维度
        - n_fft: FFT窗口大小，影响频率分辨率
        - n_mels: 中间梅尔滤波器组数量
        - hop_length: 帧移，控制时间分辨率
        - window: 窗函数类型，'hamming'提供良好的频率泄漏控制
        - center: 是否对信号进行中心填充
        - pad_mode: 填充模式，'reflect'进行镜像填充
        - power: 功率谱指数，2.0表示功率谱
        - htk: 是否使用HTK Mel刻度公式
        - fmin/fmax: 频率范围限制
        - norm: 归一化方式
        - trainable_mel/trainable_STFT: 是否允许训练过程中调整参数
    """
    # 初始化MFCC变换对象
    mfcc_transformer = Spectrogram.MFCC(
        sr=sample_rate,
        n_mfcc=n_mfcc,
        n_fft=2048,
        n_mels=128,
        hop_length=512,
        window='hamming',
        center=True,
        pad_mode='reflect',
        power=2.0,
        htk=False,
        fmin=0,
        fmax=None,
        norm=1,
        trainable_mel=False,
        trainable_STFT=False,
        verbose=False
    )
    return mfcc_transformer

In [ ]:
def save_mfcc_to_csv(mfcc_features, output_csv):
    """
    将MFCC特征保存为CSV文件
    :param mfcc_features: MFCC特征矩阵
    :param output_csv: CSV文件路径
    """
    df = pd.DataFrame(mfcc_features)
    df.to_csv(output_csv, index=False, header=False)
    print(f"MFCC特征已保存至 {output_csv}")

In [ ]:
def plot_mfcc_features(mfcc_data, sr, output_image=None):
    """
    可视化MFCC特征
    :param mfcc_data: MFCC特征矩阵
    :param sr: 采样率
    :param output_image: 如果提供路径，则保存图片
    """
    plt.figure(figsize=(12, 8))

    librosa.display.specshow(mfcc_data, sr=sr, hop_length=512, x_axis='time', cmap='turbo')
    plt.title("MFCC Features")
    plt.xlabel("Time(s)")
    plt.ylabel("MFCC Coefficients")
    plt.colorbar(format='%+2.0f dB')
    plt.tight_layout()

    if output_image:
        plt.savefig(output_image)
        print(f"MFCC特征图已保存至 {output_image}")
    else:
        plt.show()

In [ ]:
# 示例使用代码
# 假设有音频文件路径
# audio_file = 'path/to/your/audio.wav'

# 加载音频文件
# waveform, sample_rate = torchaudio.load(audio_file)

# 创建MFCC变换器
# mfcc_transformer = extract_mfcc_transformer(sample_rate, n_mfcc=13)

# 提取MFCC特征
# mfcc_features = mfcc_transformer(waveform)

# 转换为numpy数组便于处理
# mfcc_data = mfcc_features.squeeze().numpy()

# 保存MFCC特征到CSV
# save_mfcc_to_csv(mfcc_data, 'mfcc_features.csv')

# 可视化MFCC特征
# plot_mfcc_features(mfcc_data, sample_rate, 'mfcc_features.png')

print("MFCC特征提取模块已准备就绪")

In [ ]:
def extract_mfcc_with_delta(sample_rate, n_mfcc=13, include_delta=True, include_delta2=True):
    """
    提取MFCC特征及其一阶、二阶差分特征
    
    Args:
        sample_rate (int): 采样率
        n_mfcc (int): MFCC系数数量
        include_delta (bool): 是否包含一阶差分（Delta）
        include_delta2 (bool): 是否包含二阶差分（Delta-Delta）
    
    Returns:
        function: 用于提取增强MFCC特征的函数
    """
    def extract_features(waveform):
        # 基础MFCC变换器
        mfcc_transformer = extract_mfcc_transformer(sample_rate, n_mfcc)
        
        # 提取基础MFCC特征
        mfcc_features = mfcc_transformer(waveform)
        mfcc_data = mfcc_features.squeeze().numpy()
        
        features_list = [mfcc_data]
        
        if include_delta:
            # 计算一阶差分
            delta_mfcc = librosa.feature.delta(mfcc_data)
            features_list.append(delta_mfcc)
        
        if include_delta2:
            # 计算二阶差分
            delta2_mfcc = librosa.feature.delta(mfcc_data, order=2)
            features_list.append(delta2_mfcc)
        
        # 拼接所有特征
        combined_features = np.vstack(features_list)
        
        return combined_features
    
    return extract_features

In [ ]:
def compare_mfcc_configurations(waveform, sample_rate):
    """
    比较不同MFCC配置的效果
    """
    configurations = [
        {'n_mfcc': 12, 'name': 'MFCC-12'},
        {'n_mfcc': 13, 'name': 'MFCC-13'},
        {'n_mfcc': 20, 'name': 'MFCC-20'}
    ]
    
    fig, axes = plt.subplots(len(configurations), 1, figsize=(12, 4*len(configurations)))
    
    for i, config in enumerate(configurations):
        # 创建MFCC变换器
        mfcc_transformer = extract_mfcc_transformer(sample_rate, config['n_mfcc'])
        
        # 提取特征
        mfcc_features = mfcc_transformer(waveform)
        mfcc_data = mfcc_features.squeeze().numpy()
        
        # 可视化
        ax = axes[i] if len(configurations) > 1 else axes
        librosa.display.specshow(mfcc_data, sr=sample_rate, hop_length=512, 
                                x_axis='time', ax=ax, cmap='turbo')
        ax.set_title(f"{config['name']} Features")
        ax.set_ylabel('MFCC Coefficients')
        
        print(f"{config['name']} shape: {mfcc_data.shape}")
    
    plt.tight_layout()
    plt.show()

## MFCC特征提取使用说明

### 基本用法
1. 使用 `extract_mfcc_transformer()` 创建MFCC变换器
2. 使用变换器处理音频波形数据
3. 使用 `save_mfcc_to_csv()` 保存特征
4. 使用 `plot_mfcc_features()` 可视化特征

### 高级用法
- 使用 `extract_mfcc_with_delta()` 提取包含差分特征的增强MFCC
- 使用 `compare_mfcc_configurations()` 比较不同配置效果

### 参数建议
- **n_mfcc**: 通常使用12-13个系数，语音识别中13个较常见
- **n_fft**: 2048适合大多数音频分析任务
- **hop_length**: 512提供良好的时间分辨率
- **n_mels**: 128个梅尔滤波器提供足够的频率分辨率

### MFCC vs Mel频谱
- **Mel频谱**: 保留更多频率信息，适合音频分类
- **MFCC**: 更紧凑的表示，适合语音识别和说话人识别